In [1]:
import pandas as pd 
import numpy as np
import os
from math import ceil
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.rdForceFieldHelpers import UFFOptimizeMolecule
import re

In [2]:
import sys
sys.path.append('../evaluation/')

In [3]:
from utils_eval import build_pdb_dict

In [4]:
# root_path = 'pharmacophore_specific_eval/1ywn'  # 0 0 
# root_path = 'pharmacophore_specific_eval/4ara' # 0.48 0.49
# root_path = 'pharmacophore_specific_eval/1lhn'  # 0.08 0.08
# root_path = 'pharmacophore_specific_eval/4b84'  # 0.57 0.70
# root_path = 'pharmacophore_specific_eval/5ni6'  # 0.07 0.08
# root_path = 'pharmacophore_specific_eval/5idn'  # 0.23 0.25 
# root_path = 'pharmacophore_specific_eval/5ni2'  # 0.13 0.15
# root_path = 'pharmacophore_specific_eval/4ijw'  # 0.36 0.39
# root_path = 'pharmacophore_specific_eval/3ey4'  # 0.49 0.45
# root_path = 'pharmacophore_specific_eval/3r58'  # 0.70 0.70
# root_path = 'pharmacophore_specific_eval/4k1l'  # 0.58 0.62



# aromatic = True

In [174]:
# ligand_name = '1ywn_A_rec_1ywn_lif_lig_tt_docked_0'  # 0 0
# ligand_name = '4b81_B_rec_4ara_c56_lig_tt_min_0'  # 0.41 0.51
# ligand_name = '1lhu_A_rec_1lhn_aon_lig_tt_min_0'  # 0.07 0.11
# ligand_name = '4b84_A_rec_4b84_z5k_lig_tt_min_0'  # 0.60 0.61
# ligand_name = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0'  
# ligand_name = '5hbh_A_rec_5idn_6a7_lig_tt_min_0'
# ligand_name = '5aen_A_rec_5ni2_dj3_lig_tt_min_0'
# ligand_name = '3d3e_B_rec_4ijw_1eq_lig_tt_min_0'
# ligand_name = '4yyz_B_rec_3ey4_352_lig_tt_docked_12'
# ligand_name = '4h7c_A_rec_3r58_nps_lig_tt_min_0'  # 0.64 0.70
# ligand_name = '3ey4_C_rec_4k1l_sff_lig_tt_min_0'  # 0.59 0.59 

# ligand_name = '3hfg_A_rec_3hfg_17r_lig_tt_min_0'
# ligand_name = '3ey4_C_rec_5pgu_8k4_lig_tt_min_0'  # 0.11 0.10
# ligand_name = '5ni2_A_rec_5aen_dp8_lig_tt_min_0'
# ligand_name = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0'
# ligand_name = '4yyz_B_rec_2ilt_nn1_lig_tt_min_0'  # 0.16 0.20 
# ligand_name = '4bnn_A_rec_4bnm_tcu_lig_tt_min_0'
# ligand_name = '4yvx_A_rec_1s2a_imn_lig_tt_min_0'
# ligand_name = '3czr_B_rec_4bb5_hd2_lig_tt_min_0'
# ligand_name = '3czr_A_rec_2ilt_nn1_lig_tt_min_0'
# ligand_name = '4b84_A_rec_4btl_5gz_lig_tt_min_0'
# ligand_name = '3fue_A_rec_3ftv_11x_lig_tt_min_0'  # 0.04 0.05
# ligand_name = '4bnk_B_rec_4d44_ja3_lig_tt_min_0'  # 0.26 0.27
# ligand_name = '3qqp_B_rec_4k1l_sff_lig_tt_min_0'
# ligand_name = '4wdu_A_rec_3r94_flr_lig_tt_min_0'  # 0.37 0.27
# ligand_name = '4alj_A_rec_4bnf_pv4_lig_tt_docked_0'  # 0.24 0.24
# ligand_name = '1ejj_A_rec_1o99_2pg_lig_tt_docked_0'
# ligand_name = '4h96_B_rec_4h95_06u_lig_tt_min_0'  # 0.40 0.42
# ligand_name = '2oa6_D_rec_4kvi_1sv_lig_tt_min_0'  # 0.36 0.41
# ligand_name = '4hx5_B_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgy_D_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgv_A_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '4kux_B_rec_4kvw_jf4_lig_tt_min_0'  # 0.61 0.63
# ligand_name = '4bnk_B_rec_4bnl_w1g_lig_tt_min_0'
# ligand_name = '4b82_B_rec_4ara_c56_lig_tt_min_0'  # 0.42 0.53
# ligand_name = '4pm0_A_rec_4y2b_epk_lig_tt_min_0'
# ligand_name = '4ms6_A_rec_3fuh_5h1_lig_tt_min_0'  # 0.02 0.02
# ligand_name = '5ivg_B_rec_4kvy_jf1_lig_tt_docked_9'  # 0.61 0.69
# ligand_name = '3r8h_A_rec_3r8h_zom_lig_tt_min_0'
# ligand_name = '1aoe_A_rec_3qlr_qlr_lig_tt_docked_4'  # 0.19 0.28
# ligand_name = '2ilt_A_rec_3ey4_352_lig_tt_min_0'  # 0.70 0.71
# ligand_name = '2g1q_A_rec_1x88_nat_lig_tt_min_0'  # 0.59 0.59
# ligand_name = '5imp_B_rec_4kwd_jf2_lig_tt_min_0'  # 0.54 0.55
ligand_name = '4d45_A_rec_4ali_tcl_lig_tt_min_0'  # 0.57 0.52

# folder_name = ligand_name[ligand_name.rfind('rec')+4:ligand_name.rfind('rec')+8]
root_path = os.path.join('structure_based', ligand_name)

aromatic = False

In [175]:
log_path = os.path.join(root_path, 'logs')
log_path = log_path+'_aromatic' if aromatic else log_path

raw_data_path = '../../data/cleaned_crossdocked_data/raw'

log_files = os.listdir(log_path)
pdb_dict, pdb_rev_dict = build_pdb_dict(raw_data_path)

In [176]:
def get_minimized_affinity(filename, mode='ref'):
    if mode == 'ref':
        with open(filename, 'r') as file:
            for line in file:
                if '<minimizedAffinity>' in line:
                    # The next line after '<minimizedAffinity>' contains the value
                    return float(next(file))
            
    else:
        with open(filename, 'r') as f:
            lines = f.readlines()
            lines = lines[19:28]
#             print(len(lines))
            n = len(lines)
            arr = np.zeros(n)
            cnn_arr = np.zeros(n)
            print(filename)
            for i in range(n):
                arr[i] = float(lines[i][11:17])
                cnn_arr[i] = float(lines[i][34:42])
#             print(arr)
            try:
                score = np.min(arr)
                cnn_score = np.min(cnn_arr)
            except:
                score = None
                cnn_score = None
                print(f'{filename} failed')
                print('its scores: ', arr)
        return score, cnn_score

In [177]:
def get_affinity_best_pose(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        lines = lines[19:28]
        try:
            score = float(lines[0][11:17])
            cnn_score = float(lines[0][34:42])
            # cnn_score = float(lines[0][24:30])
        except:
            score = None
            cnn_score = None
        return score, cnn_score

In [178]:
# ligand_filename = '1ywn_A_rec_1ywn_lif_lig_tt_docked_0.sdf'
# ligand_filename = '4b81_B_rec_4ara_c56_lig_tt_min_0.sdf'
# ligand_filename = '1lhu_A_rec_1lhn_aon_lig_tt_min_0.sdf'
# ligand_filename = '4b84_A_rec_4b84_z5k_lig_tt_min_0.sdf'
# ligand_filename = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0.sdf'
# ligand_filename = '5hbh_A_rec_5idn_6a7_lig_tt_min_0.sdf'
# ligand_filename = '5aen_A_rec_5ni2_dj3_lig_tt_min_0.sdf'
# ligand_filename = '3d3e_B_rec_4ijw_1eq_lig_tt_min_0.sdf'
# ligand_filename = '4yyz_B_rec_3ey4_352_lig_tt_docked_12.sdf'
# ligand_filename = '4h7c_A_rec_3r58_nps_lig_tt_min_0.sdf'
# ligand_filename = '3ey4_C_rec_4k1l_sff_lig_tt_min_0.sdf'

ligand_filename = ligand_name + '.sdf'
pdb_folder = pdb_rev_dict[ligand_filename]
ref_file = os.path.join(raw_data_path, pdb_folder, ligand_filename)
ref_score = get_minimized_affinity(ref_file, 'ref')

score_dict = {}
for log_file in tqdm(log_files):
    filename = log_file.split('.')[0]
#     print(log_file)
    gen_score, cnn_score = get_minimized_affinity(os.path.join(log_path, log_file), 'gen')
    # gen_score, cnn_score = get_affinity_best_pose(os.path.join(log_path, log_file))
    score_dict[filename] = [gen_score, cnn_score, ref_score]

 23%|████████████████████████████▋                                                                                               | 105/454 [00:00<00:00, 524.92it/s]

structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_431.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_131.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_270.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_168.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_221.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_453.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_64.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_480.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_416.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_155.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_166.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_102.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_263.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_372.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_380.

 35%|███████████████████████████████████████████▏                                                                                | 158/454 [00:00<00:00, 519.82it/s]

structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_5.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_462.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_468.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_386.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_490.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_193.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_322.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_136.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_37.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_343.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_329.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_50.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_204.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_382.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_199.log

 58%|████████████████████████████████████████████████████████████████████████▍                                                   | 265/454 [00:00<00:00, 527.74it/s]

structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_285.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_290.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_422.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_39.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_281.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_414.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_459.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_383.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_336.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_344.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_291.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_282.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_149.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_196.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_23.l

 82%|█████████████████████████████████████████████████████████████████████████████████████████████████████▎                      | 371/454 [00:00<00:00, 518.96it/s]

structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_393.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_0.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_205.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_11.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_127.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_469.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_63.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_228.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_133.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_177.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_496.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_217.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_148.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_370.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_114.log

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 454/454 [00:00<00:00, 522.98it/s]

structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_69.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_235.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_401.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_316.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_324.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_55.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_357.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_481.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_287.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_158.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_82.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_394.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_475.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_117.log
structure_based/4d45_A_rec_4ali_tcl_lig_tt_min_0/logs/4ali_41.log

In [179]:
df = pd.DataFrame.from_dict(score_dict, orient='index', columns=['Binding affinity', 'CNN score', 'Reference'])
df

,Binding affinity,CNN score,Reference
4ali_431,-9.31,0.3227,-7.26438
4ali_131,-6.39,0.6701,-7.26438
4ali_270,-6.95,0.5354,-7.26438
4ali_168,-6.42,0.5072,-7.26438
4ali_221,-8.45,0.3693,-7.26438
...,...,...,...
4ali_242,-6.90,0.4691,-7.26438
4ali_252,-6.55,0.3654,-7.26438
4ali_464,-6.60,0.4370,-7.26438
4ali_27,-5.11,0.3821,-7.26438


In [180]:
filtered_df = df.loc[df['Binding affinity']<df['Reference']]
filtered_df

,Binding affinity,CNN score,Reference
4ali_431,-9.31,0.3227,-7.26438
4ali_221,-8.45,0.3693,-7.26438
4ali_453,-7.82,0.5003,-7.26438
4ali_480,-9.28,0.3637,-7.26438
4ali_166,-7.48,0.3373,-7.26438
...,...,...,...
4ali_278,-7.44,0.3709,-7.26438
4ali_115,-8.46,0.4662,-7.26438
4ali_301,-7.28,0.4247,-7.26438
4ali_477,-8.47,0.4466,-7.26438


In [181]:
sorted_df = filtered_df.sort_values(by='CNN score', ascending=False)
sorted_df

,Binding affinity,CNN score,Reference
4ali_195,-7.69,0.6386,-7.26438
4ali_177,-7.28,0.6004,-7.26438
4ali_486,-7.81,0.6003,-7.26438
4ali_265,-7.47,0.6000,-7.26438
4ali_37,-7.49,0.5979,-7.26438
...,...,...,...
4ali_112,-9.23,0.2594,-7.26438
4ali_186,-8.30,0.2533,-7.26438
4ali_121,-7.87,0.2216,-7.26438
4ali_281,-9.15,0.2136,-7.26438


In [182]:
ba_res_filename = os.path.join(root_path, 'binding_affinity')
ba_res_filename = ba_res_filename+'_aromatic.csv' if aromatic else ba_res_filename+'.csv'

# sorted_df.to_csv(ba_res_filename)
# ba_res_filename

In [183]:
pp_match_csv = os.path.join(root_path, 'aromatic_scores.csv' if aromatic else 'basic_scores.csv')
pp_match_df = pd.read_csv(pp_match_csv, header=0, index_col=0)
pp_match_df = pp_match_df.rename(columns = {'score':'PP match score'})
pp_match_df

,PP match score
4ali_89,0.285714
4ali_64,0.142857
4ali_85,0.571429
4ali_413,0.142857
4ali_280,0.428571
...,...
4ali_8,0.428571
4ali_428,0.428571
4ali_145,0.142857
4ali_379,0.285714


In [184]:
joint_df = df.join(pp_match_df)
joint_df

,Binding affinity,CNN score,Reference,PP match score
4ali_431,-9.31,0.3227,-7.26438,0.285714
4ali_131,-6.39,0.6701,-7.26438,0.428571
4ali_270,-6.95,0.5354,-7.26438,0.285714
4ali_168,-6.42,0.5072,-7.26438,0.285714
4ali_221,-8.45,0.3693,-7.26438,0.428571
...,...,...,...,...
4ali_242,-6.90,0.4691,-7.26438,0.285714
4ali_252,-6.55,0.3654,-7.26438,0.571429
4ali_464,-6.60,0.4370,-7.26438,0.285714
4ali_27,-5.11,0.3821,-7.26438,0.285714


In [185]:
joint_res_fn = 'joint_res_aromatic.csv' if aromatic else 'joint_res.csv'
joint_df.to_csv(os.path.join(root_path, joint_res_fn))

In [186]:
sum(df['Binding affinity'] <= df['Reference']) / len(df)

0.5682819383259912